**Проект для «Викишоп» c ST и BERT**

**Описание проекта:** 
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75. 


**Цель исследования:** Обучить модель классифицировать комментарии на позитивные и негативные. 

**Ход исследования:**
- предобработка данных
- создание эмбеддингов
- обучение моделей: LR, RFC, LGMBC
- определение лучшей и подсчет метрики F1 на лучшей модели


In [1]:
%pip install scikit-learn -q -U 
%pip install sentence-transformers lightgbm optuna transformers tqdm -q
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import warnings
import torch
import optuna

from typing import List

from tqdm import tqdm

from sentence_transformers import SentenceTransformer
from transformers.tokenization_utils_base import BatchEncoding
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from lightgbm import LGBMClassifier
from lightgbm import early_stopping

In [3]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
BATCH_SIZE = 32

In [4]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

## Первичная предобработка данных

In [5]:
data = pd.read_csv(r'../datasets/toxic_comments.csv')

data.info()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [6]:
data = data.drop('Unnamed: 0', axis=1)

In [7]:
data.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [8]:
data.duplicated().sum()

0

In [9]:
print(data['toxic'].value_counts())
print()
print(data['toxic'].value_counts(normalize=True))

0    143106
1     16186
Name: toxic, dtype: int64

0    0.898388
1    0.101612
Name: toxic, dtype: float64


Проверили данные на дубликаты и избавились от проблемного столбца, который отвечал за индексацию. (в нем не все значения идут по порядку)<br>
Также можем наблюдать сильный дисбаланс в классах. Одного класса практически 90% от всей выборки.

In [10]:
def stratified_sample(df: pd.DataFrame, target_col: str, total_n: int) -> pd.DataFrame:
    return (
        df
        .groupby(target_col, group_keys=False)
        .apply(lambda x: x.sample(int(len(x) / len(df) * total_n), random_state=RANDOM_STATE))
        .reset_index(drop=True)
    )

In [11]:
mini_df = stratified_sample(data, 'toxic', 2_501)
print(mini_df['toxic'].value_counts(normalize=True))
mini_df.shape

0    0.8984
1    0.1016
Name: toxic, dtype: float64


(2500, 2)

Так как проект для учебных целей сократим выборку с сохранением пропорций классов, чтобы энергозатратные модели отработали быстрее

## Применение модели SentenceTransformer

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Используемое устройство: {device}')

Используемое устройство: cuda


In [13]:
model_st = SentenceTransformer("all-MiniLM-L6-v2", device=device)

embeddings = model_st.encode(
                    mini_df['text'].to_list(),
                    convert_to_numpy=True,
                    normalize_embeddings=True,
                    show_progress_bar=True
                    )

print(f'Форма эмбеддингов: {embeddings.shape}')

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Форма эмбеддингов: (2500, 384)


In [14]:
temp_df = pd.DataFrame(
    np.hstack((embeddings, mini_df['toxic'].to_numpy() \
                                        .reshape(-1, 1)))
    )
print(f'Количество дубликатов: {temp_df.duplicated().sum()}')
temp_df.drop_duplicates(inplace=True)
print(f'Количество дубликатов после удаления: {temp_df.duplicated().sum()}')

X_clean = temp_df.iloc[:, :-1].to_numpy()
y_clean = temp_df.iloc[:, -1].to_numpy()

Количество дубликатов: 0
Количество дубликатов после удаления: 0


In [15]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_clean, y_clean, 
    random_state=RANDOM_STATE, 
    stratify=y_clean, test_size=TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, 
    random_state=RANDOM_STATE, 
    stratify=y_train_full, test_size=TEST_SIZE)

In [16]:
print(X_train.shape, X_test.shape, X_val.shape)

(1600, 384) (500, 384) (400, 384)


### Обучение моделей

#### LR

In [17]:
def obj_logreg(trial):
    params = {
        'random_state': RANDOM_STATE,
        'C': trial.suggest_float('C', 0.1, 2)
    }

    model = LogisticRegression(**params).fit(X_train, y_train)

    y_preds = model.predict(X_val)
    f1 = f1_score(y_val, y_preds, average='binary')
    return f1

In [18]:
study_logreg = optuna.create_study(direction='maximize')
study_logreg.optimize(obj_logreg, n_trials=10)

[I 2025-07-05 21:36:50,580] A new study created in memory with name: no-name-b2ee461f-7d97-48b4-b652-7c10ab9e5c4e
[I 2025-07-05 21:36:50,602] Trial 0 finished with value: 0.6333333333333333 and parameters: {'C': 1.480688432722877}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-07-05 21:36:50,626] Trial 1 finished with value: 0.6101694915254238 and parameters: {'C': 1.305990380470312}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-07-05 21:36:50,642] Trial 2 finished with value: 0.5614035087719298 and parameters: {'C': 0.9102678223617086}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-07-05 21:36:50,658] Trial 3 finished with value: 0.09302325581395349 and parameters: {'C': 0.24600445978795052}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-07-05 21:36:50,671] Trial 4 finished with value: 0.4230769230769231 and parameters: {'C': 0.5656964163128269}. Best is trial 0 with value: 0.6333333333333333.
[I 2025-07-05 21:36:50,688] Trial 5 finished with v

#### RFC

In [19]:
def obj_rfc(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'class_weight': 'balanced',
        'criterion': trial.suggest_categorical('criterion', ['log_loss', 'gini', 'entropy']),
        'random_state': RANDOM_STATE,
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 25),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_depth': trial.suggest_int('max_depth', 10, 40),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
    }

    model = RandomForestClassifier(**params).fit(X_train, y_train)

    preds = model.predict(X_val)

    return f1_score(y_val, preds, average='binary')

In [20]:
study_rfc = optuna.create_study(direction='maximize')
study_rfc.optimize(obj_rfc, n_trials=25)

[I 2025-07-05 21:36:50,824] A new study created in memory with name: no-name-65a8a808-22ae-4781-a2c5-a13f67d8d84c
[I 2025-07-05 21:36:51,919] Trial 0 finished with value: 0.17777777777777778 and parameters: {'n_estimators': 101, 'criterion': 'log_loss', 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_depth': 40, 'max_features': 'log2'}. Best is trial 0 with value: 0.17777777777777778.
[I 2025-07-05 21:36:54,475] Trial 1 finished with value: 0.38461538461538464 and parameters: {'n_estimators': 124, 'criterion': 'log_loss', 'min_samples_split': 14, 'min_samples_leaf': 8, 'max_depth': 20, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.38461538461538464.
[I 2025-07-05 21:36:56,628] Trial 2 finished with value: 0.2857142857142857 and parameters: {'n_estimators': 102, 'criterion': 'entropy', 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_depth': 35, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.38461538461538464.
[I 2025-07-05 21:36:59,490] Trial 3 finished with va

#### LGBMC

In [21]:
def obj_lgbmc(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'random_state': RANDOM_STATE,
        'n_estimators': 10000, 
        'verbose': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0)
    }

    model = LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(stopping_rounds=100)],
    )
    preds = model.predict(X_val)

    return f1_score(y_val, preds, average='binary')



In [22]:
study_lgbmc = optuna.create_study(direction='maximize')
study_lgbmc.optimize(obj_lgbmc, n_trials=25)

[I 2025-07-05 21:37:33,854] A new study created in memory with name: no-name-21034ecf-73ff-483d-9062-f0a4b5d84eca


Training until validation scores don't improve for 100 rounds


[I 2025-07-05 21:37:34,736] Trial 0 finished with value: 0.6129032258064516 and parameters: {'learning_rate': 0.2964428483809943, 'num_leaves': 887, 'max_depth': 13, 'min_child_samples': 70, 'subsample': 0.46878563849778887, 'colsample_bytree': 0.4087034875590403}. Best is trial 0 with value: 0.6129032258064516.


Early stopping, best iteration is:
[57]	valid_0's auc: 0.920443
Training until validation scores don't improve for 100 rounds


[I 2025-07-05 21:37:36,147] Trial 1 finished with value: 0.5517241379310345 and parameters: {'learning_rate': 0.044079906937748506, 'num_leaves': 173, 'max_depth': 13, 'min_child_samples': 99, 'subsample': 0.9094544712763762, 'colsample_bytree': 0.5293593591015506}. Best is trial 0 with value: 0.6129032258064516.


...

[I 2025-07-05 21:38:08,810] Trial 22 finished with value: 0.6229508196721312 and parameters: {'learning_rate': 0.2143938987851304, 'num_leaves': 2561, 'max_depth': 14, 'min_child_samples': 60, 'subsample': 0.9276473302618113, 'colsample_bytree': 0.5581082025258487}. Best is trial 22 with value: 0.6229508196721312.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[78]	valid_0's auc: 0.904545


[I 2025-07-05 21:38:10,647] Trial 23 finished with value: 0.5 and parameters: {'learning_rate': 0.20698451633742382, 'num_leaves': 2624, 'max_depth': 14, 'min_child_samples': 61, 'subsample': 0.8790194412382781, 'colsample_bytree': 0.5746419481325565}. Best is trial 22 with value: 0.6229508196721312.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[110]	valid_0's auc: 0.912018


[I 2025-07-05 21:38:12,844] Trial 24 finished with value: 0.5806451612903226 and parameters: {'learning_rate': 0.23143639751001902, 'num_leaves': 2367, 'max_depth': 14, 'min_child_samples': 76, 'subsample': 0.9249220355981248, 'colsample_bytree': 0.7037134496964397}. Best is trial 22 with value: 0.6229508196721312.


In [23]:
print(f'LR: {study_logreg.best_value:.4f}')
print(f'RFC: {study_rfc.best_value:.4f}')
print(f'LGBM: {study_lgbmc.best_value:.4f}')

LR: 0.6333
RFC: 0.6364
LGBM: 0.6230


Как мы можем заметить, модели обученные на эмбеддингах от SentenceTransformer отработали не лучшем образом. Их метрика f1 ниже необходимого 0.75. Вероятнее всего, проблема в том, что данная модель обучалась на задачах, специфичных для семантического сходства предложений (Natural Language Inference, Semantic Textual Similarity), что делает эмбеддинги более подходящими для сравнения предложений, а не классификации тональности сообщений.<br><br>
Поэтому создадим эмбеддинги с помощью дообученной модели BERT.

## Токенизация и применение модели BERT

In [24]:
#Если есть GPU, то создаем объект device и прописываем его равным cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемое устройство:', device)

Используемое устройство: cuda


In [25]:
tokenizer = AutoTokenizer.from_pretrained('unitary/toxic-bert')
model_bert = AutoModel.from_pretrained('unitary/toxic-bert')

model_bert.to(device) # переключаем модель на GPU, если он есть
model_bert.eval() # переключаем модель для инференса

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [26]:
#Класс для преобразования списка строк в специальный объект для DataLoader
class TextDataset(Dataset):
    def __init__(self, text: List[str]) -> None:
        self.text = text

    def __len__(self) -> int:
        return len(self.text)
    
    def __getitem__(self, index: int) -> str:
        return self.text[index]

def tokenize(batch: List[str]) -> BatchEncoding:
    return tokenizer(
        batch,
        return_tensors='pt',
        padding=True,
        truncation=True
    )

def get_embeddings(text: List[str], batch_size: int = BATCH_SIZE) -> np.ndarray:
    dataset = TextDataset(text)
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=tokenize)

    all_embeddings = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Извлечение эмбеддингов'):
            batch = batch.to(device)
            embeddings = model_bert(**batch).last_hidden_state[:, 0, :]

            if device.type == 'cuda':
                all_embeddings.append(embeddings.cpu())
            else: 
                all_embeddings.append(embeddings)
    
    return torch.cat(all_embeddings, dim=0).numpy()

In [27]:
embeddings_bert = get_embeddings(mini_df['text'])

print('Форма эмбеддингов BERT', embeddings_bert.shape)

Извлечение эмбеддингов: 100%|██████████| 79/79 [02:15<00:00,  1.71s/it]

Форма эмбеддингов BERT (2500, 768)


In [28]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
                                                embeddings_bert, mini_df['toxic'],
                                                random_state=RANDOM_STATE, 
                                                stratify=mini_df['toxic'], test_size=TEST_SIZE)
X_train, X_val, y_train, y_val = train_test_split(
                                    X_train_full, y_train_full,
                                    random_state=RANDOM_STATE,
                                    stratify=y_train_full, test_size=TEST_SIZE)

### Обучение моделей

In [29]:
study_logreg_bert = optuna.create_study(direction='maximize')
study_logreg_bert.optimize(obj_logreg, n_trials=10)

study_rfc_bert = optuna.create_study(direction='maximize')
study_rfc_bert.optimize(obj_rfc, n_trials=40)

study_lgbmc_bert = optuna.create_study(direction='maximize')
study_lgbmc_bert.optimize(obj_lgbmc, n_trials=40)

[I 2025-07-05 21:40:29,497] A new study created in memory with name: no-name-8f21abcb-8555-4343-80ff-0a0d26ee0ce5
[I 2025-07-05 21:40:29,597] Trial 0 finished with value: 0.9411764705882353 and parameters: {'C': 0.37826947792983645}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-07-05 21:40:29,648] Trial 1 finished with value: 0.9411764705882353 and parameters: {'C': 0.4157276879938434}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-07-05 21:40:29,701] Trial 2 finished with value: 0.9411764705882353 and parameters: {'C': 1.3279524711005537}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-07-05 21:40:29,762] Trial 3 finished with value: 0.9411764705882353 and parameters: {'C': 1.8456196387850619}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-07-05 21:40:29,821] Trial 4 finished with value: 0.9411764705882353 and parameters: {'C': 1.9531960411725946}. Best is trial 0 with value: 0.9411764705882353.
[I 2025-07-05 21:40:29,888] Trial 5 finished with 

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[19]	valid_0's auc: 0.998098


[I 2025-07-05 21:41:09,015] Trial 0 finished with value: 0.9523809523809523 and parameters: {'learning_rate': 0.2634543850813775, 'num_leaves': 1347, 'max_depth': 10, 'min_child_samples': 58, 'subsample': 0.9501748218594166, 'colsample_bytree': 0.9557901608980008}. Best is trial 0 with value: 0.9523809523809523.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[43]	valid_0's auc: 0.998369


[I 2025-07-05 21:41:11,550] Trial 1 finished with value: 0.9411764705882353 and parameters: {'learning_rate': 0.1263274979911734, 'num_leaves': 2161, 'max_depth': 9, 'min_child_samples': 86, 'subsample': 0.8038516859766771, 'colsample_bytree': 0.5693906357726732}. Best is trial 0 with value: 0.9523809523809523.


...

Early stopping, best iteration is:
[14]	valid_0's auc: 0.997962
Training until validation scores don't improve for 100 rounds


[I 2025-07-05 21:42:36,677] Trial 38 finished with value: 0.9397590361445783 and parameters: {'learning_rate': 0.1428942141058671, 'num_leaves': 843, 'max_depth': 7, 'min_child_samples': 83, 'subsample': 0.9909313486730564, 'colsample_bytree': 0.8865733273281082}. Best is trial 7 with value: 0.9647058823529412.


Early stopping, best iteration is:
[32]	valid_0's auc: 0.998641
Training until validation scores don't improve for 100 rounds


[I 2025-07-05 21:42:38,319] Trial 39 finished with value: 0.9397590361445783 and parameters: {'learning_rate': 0.27793877039612064, 'num_leaves': 650, 'max_depth': 11, 'min_child_samples': 61, 'subsample': 0.6115989006527659, 'colsample_bytree': 0.9091416657044236}. Best is trial 7 with value: 0.9647058823529412.


Early stopping, best iteration is:
[21]	valid_0's auc: 0.997282


In [30]:
print(f'Метрика f1 на модели LR: {study_logreg_bert.best_value:.4f}')
print(f'Метрика f1 на модели RFC: {study_rfc_bert.best_value:.4f}')
print(f'Метрика f1 на модели LGBMC: {study_lgbmc_bert.best_value:.4f}')

Метрика f1 на модели LR: 0.9412
Метрика f1 на модели RFC: 0.9762
Метрика f1 на модели LGBMC: 0.9647


Лучше всего на валидационной выборке показала себя модель Random Forest. Метрика f1 равна 0.9762.

## Предсказание на тестовой выборке

In [32]:
model_final = RandomForestClassifier(**study_rfc_bert.best_params).fit(X_train, y_train)

test_preds = model_final.predict(X_test)

f1_final = f1_score(y_test, test_preds, average='binary')
print(f'Метрика f1 на тестовой выборке {f1_final:.4f}')

Метрика f1 на тестовой выборке 0.9434


Метрика f1 на тестовой выборке больше 0.7. Задачу можно считать выполненной. Метрика лучшей модели равна: 0.94

## Вывод

- Были загружены и обработаны данные
- Текста преобразованы в эмбеддинги с помощью SentenceTransformer. Обучены три модели: LR, RFC, LGBMC. Метрика на валидационнной выборке неудовлетворительная.
- Токенизировали и преобразовали данные с помощью модели BERT. Те же самые модели были обучены. Все три модели на валидационной выборке получили метрику выше 0.75. Лучше всего показала себя модель Random Forest.
- Предсказана метрика на тестовой выборке с помощью модели Random Forest. F1 равно 0.94, что больше 0.75